In [ ]:
import pandas as pd
import numpy as np
import folium
import requests

from bs4 import BeautifulSoup

# pip install tqdm : progressBar 구현
# from tqdm.notebook import tqdm
from tqdm import tqdm
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# jupyter nbconvert --to script coffeeStore.ipynb

In [ ]:
# 크롬 드라이버 다운로드 :  https://googlechromelabs.github.io/chrome-for-testing/
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간 10초
driver.implicitly_wait(wait_time)

In [ ]:
# driver.maximize_window()

In [ ]:
# 공용 변수 선언
dataOut = './../dataOut/'

In [ ]:
# 스타벅스) 매장 찾기-지역 검색 클릭
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(starbucks_url) # 해당 페이지로 이동하기

In [ ]:
# 스타벅스 '서울' 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector))).click()

In [ ]:
# 스타벅스 '서울'-'전체' 링크 클릭
starbucks_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all))).click()

In [ ]:
# 스타벅스 html 코드를 파싱해서 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지 소스 코드 반환
filename = 'starbucks.html'
htmlFile = open(filename, mode='wt', encoding='UTF-8')
print(html, file=htmlFile)
htmlFile.close()
print(filename, '파일 생성됨')

In [ ]:
# 파싱된 결과를 BeautifulSoup 객체로 생성
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

In [ ]:
# id 속성이 mCSB_3_container인 <div> 태그를 찾아 주세요.
container = soup.find('div', attrs={'id': 'mCSB_3_container'})

# 그 아래에 있는 모든 <li> 태그를 찾아 주세요.
storeAll = container.find_all('li')

print(f'모든 매장 갯수 : {len(storeAll)}')

In [ ]:
starbucksData = [] # 스타벅스 매장 리스트

for store in storeAll:
    # print(store)
    brand = '스타벅스'
    name = store.find('i').text.strip()
    address = store.find('p').text.replace('1522-3232', '')
    imsi = address.split(' ')
    sido = imsi[0]
    gungu = imsi[1]
    latitude = store['data-lat']
    longitude = store['data-long']

    starbucksData.append((brand, name, address, sido, gungu, latitude, longitude))
    # break
# end for

print(f'스타벅스 매장 갯수 : {len(starbucksData)}')
# print(starbucksData)

In [ ]:
'''
<li class="quickResultLstCon"
    data-code="3762"
    data-hlytag="null"
    data-index="0"
    data-lat="37.501087"
    data-long="127.043069"
    data-name="역삼아레나빌딩"
    data-storecd="1509"
    style="background:#fff">

    <strong data-my_siren_order_store_yn="N"
            data-name="역삼아레나빌딩"
            data-store="1509"
            data-yn="N">
        역삼아레나빌딩
    </strong>

    <p class="result_details">
        서울특별시 강남구 언주로 425 (역삼동)<br/>
        1522-3232
    </p>

    <i class="pin_general">리저브 매장 2번</i>

</li>

'''

In [ ]:
sbColumns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
sbDataFrame = pd.DataFrame(starbucksData)
sbDataFrame.columns = sbColumns
sbDataFrame.head()

In [ ]:
sbFilename = dataOut + 'starbucks_file.csv'
sbDataFrame.to_csv(sbFilename, index=False, encoding='UTF-8')
print(f'{sbFilename} 파일이 저장되었습니다.')

In [ ]:
# 위도, 경도 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {sbDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {sbDataFrame["경도"].isnull().sum()}')

In [ ]:
print('# 서울시 구 갯수 파악')
seoul_gu_list = list(sbDataFrame['군구'].unique())
print(f'서울시 구 갯수: {len(seoul_gu_list)}')
print(f'\n서울시 구 목록 : {seoul_gu_list}')

In [61]:
# 누락된 위도와 경도 정보는 kakao api를 이용하여 채워 넣도록 합니다.
# https://developers.kakao.com/
# kakao에 로그인하여 API 키 발급 받기
# 주의) 카카오 개발자 사이트에서 카카오맵-사용설정-On 상태로 변경해주세요.

In [77]:
url_header = 'https://dapi.kakao.com/v2/local/search/address.json?query='
api_key = 'a27595520f0c90739026b9f6ace4279c'
header = {'Authorization': 'KakaoAK ' + api_key}

In [90]:
# 주소를 입력 받아서 위도와 경도를 반환해주는 함수 구현하기
def getGeoCoder(address):
    result = ''
    url = url_header + address
    response = requests.get(url, headers=header)

    if response.status_code == 200:
        try:
            # print(response.json())
            jsondata = response.json()["documents"][0]["address"]
            return jsondata['y'], jsondata['x']
        except Exception as err:
            print(err)
            return None
        # end try
    else:
        result = f'Error[{response.status_code}]'
    # end if

    return result
# end def getGeoCoder

In [91]:
# 다음 주소에 대한 테스트를 진행합니다.
address01 = '서울 중랑구 망우로 460 (망우동)'
geoInfo01 = getGeoCoder(address01) # 잘 동작하는 주소
print(geoInfo01)

address02 = '서울 노원구 한글비석로 409 (상계동) 1~2층'
geoInfo02 = getGeoCoder(address02) # NoneType이 리턴되는 주소
geoInfo02

('37.6001065609187', '127.103136691889')
list index out of range


In [58]:
'''
케이스 01
panLatTo('127.079680860838','37.5879908042663','6');fnMove();

케이스 02
panLatTo('0','0','11');fnMove();

케이스 03
panAddTo('서울 중랑구 신내로 211 (신내동)','8');fnMove();
'''


"\n케이스 01\npanLatTo('127.079680860838','37.5879908042663','6');fnMove();\n\n케이스 02\npanLatTo('0','0','11');fnMove();\n\n케이스 03\npanAddTo('서울 중랑구 신내로 211 (신내동)','8');fnMove();\n"

In [ ]:
# 이디야 매장 정보
ediya_url = 'https://www.ediya.com/contents/find_store.html'
driver.get(ediya_url)

In [ ]:
# 이디야 '매장명'과 '주소' 검색에서 '주소' 버튼 클릭
ediya_address_selector = '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_address_selector))).click()

In [92]:
# 서울시 구 목록을 순환하면서 크롤링
ediyaData = [] # 이디야 매장 정보 리스트
incorrect_address = [] # 위도/경도 추출에 문제가 있는 주소 리스트

ediya_search_inputbox = '#keyword' # 이디야 주소 검색 입력란
ediya_search_button = '#keyword_div > form > button' # 이디야 주소 검색 (돋보기) 버튼

for gu in tqdm(seoul_gu_list):
    # 주소 검색 입력란 초기화
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).clear()

    # 주소 검색어 입력 : 예시) 서울 강남구
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).send_keys(f'서울 {gu}')

    # 찾기(돋보기) 버튼 클릭
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_button))).click()

    # 이디야 매장 정보 수집
    html = driver.page_source

    if gu == '마포구': # 하나의 구 정보만 저장해보기
        gufile = open(f'서울 {gu}.html', 'wt', encoding='UTF-8')
        print(html, file=gufile)
        gufile.close()
        print(f'서울 {gu}.html 파일 생성 완료')
    # end if

    soup = BeautifulSoup(html, 'html.parser')

    # 매장 정보는 id 속성이 placesList인 <ul> 태그 아래에 있습니다.
    ul_tag = soup.find('ul', id='placesList')

    oneGuStoreList = ul_tag.find_all('li') # 구별 매장 리스트

    for store in oneGuStoreList:
        # print(store)
        brand = '이디야'
        name = store.find('dt').text.strip()
        address = store.find('dd').text
        imsi = address.split(' ')
        sido = imsi[0]
        gungu = imsi[1]

        # 위도와 경도는 잠시
        geoInfoString = store.find('a')['onclick']
        # print(geoInfoString)

        latitude = np.nan # 결측치
        longitude = np.nan

        # 위도와 경도 보정
        if geoInfoString.startswith('panLatTo') :
            if geoInfoString.startswith("panLatTo('0','0'") : # 잘못된 위도/경도
                # address를 사용하여 Kakao API에서 물어 보기
                geo_info = getGeoCoder(address)
                if geo_info != None:
                    latitude, longitude = geo_info
                else:
                    incorrect_address.append(address)
                    latitude = None
                    longitude = None
                # end if
            else: # 올바른 위도/경도
                latLong = geoInfoString.split("'")
                latitude = latLong[3]
                longitude = latLong[1]
                # print(f'위도 : {latitude}, 경도 : {longitude}')
            # end if

        else: # 'panAddTo'으로 시작하는 경우
            # address를 사용하여 Kakao API에서 물어 보기
            geo_info = getGeoCoder(address)
            if geo_info != None:
                latitude, longitude = geo_info
            else:
                incorrect_address.append(address)
                latitude = None
                longitude = None
            # end if
        # end if

        ediyaData.append((brand, name, address, sido, gungu, latitude, longitude))

        # break # 주의) 차후 삭제 예정
    # end inner for

    # break # 주의) 차후 삭제 예정
# end outer for

print(f'이디야 매장 갯수 : {len(ediyaData)}')

  0%|          | 0/25 [00:00<?, ?it/s]

위도 : 37.6006721423547, 경도 : 127.108627207598
위도 : 37.5896269575279, 경도 : 127.094182772191
위도 : 37.5974674047065, 경도 : 127.09415879594557
위도 : 37.5992876153903, 경도 : 127.092756577852
위도 : 37.6097411262067, 경도 : 127.076897926489
위도 : 37.574252177368244, 경도 : 127.08048931056359
위도 : 37.5879908042663, 경도 : 127.079680860838
위도 : 37.5901934659094, 경도 : 127.086826066156
위도 : 37.5834835948977, 경도 : 127.083196727099
위도 : 37.5960553404392, 경도 : 127.08342099906
위도 : 37.6002908828924, 경도 : 127.087790157773
위도 : 37.60237451498618, 경도 : 127.09544803898843
위도 : 37.60748470605642, 경도 : 127.10675146670297
위도 : 37.6143693114243, 경도 : 127.078289365778


  4%|▍         | 1/25 [00:02<00:59,  2.48s/it]

위도 : 37.5993838458016, 경도 : 127.079542626772
위도 : 37.61954020149987, 경도 : 127.08413727538176
위도 : 37.51654171724045, 경도 : 127.0401601992311
위도 : 37.51408005446769, 경도 : 127.02810578707652
위도 : 37.50133876179308, 경도 : 127.05242928262568
위도 : 37.4735487656314, 경도 : 127.107863694133
위도 : 37.488604436046536, 경도 : 127.06742226515867
위도 : 37.49418891219431, 경도 : 127.07764131137398
위도 : 37.4940924880388, 경도 : 127.061617564506
위도 : 37.50043704079942, 경도 : 127.06310437319587
위도 : 37.50346806209728, 경도 : 127.05088702698811
위도 : 37.517782772275645, 경도 : 127.02145819084597
위도 : 37.527994487818226, 경도 : 127.03073566616045
위도 : 37.50178663316214, 경도 : 127.03523956508853
위도 : 37.49854753271711, 경도 : 127.04798299548432
위도 : 37.5003154494517, 경도 : 127.04292899982501


  8%|▊         | 2/25 [00:06<01:12,  3.15s/it]

...

위도 : 37.58980550001833, 경도 : 127.05749583678211
위도 : 37.5641084405355, 경도 : 127.053432156481
위도 : 37.5501670203702, 경도 : 127.04361144463
위도 : 37.54487737059238, 경도 : 127.06190839848826


 96%|█████████▌| 24/25 [01:13<00:03,  3.15s/it]

위도 : 37.53494410336005, 경도 : 127.00692431290224
위도 : 37.53584882357322, 경도 : 126.9611373908156
위도 : 37.5337400268036, 경도 : 126.956231240406
위도 : 37.5325054872953, 경도 : 126.951642152118
위도 : 37.5254293060308, 경도 : 126.964651687385
위도 : 37.5494639189725, 경도 : 126.969413497587
위도 : 37.540537976136086, 경도 : 126.96220762686157


100%|██████████| 25/25 [01:15<00:00,  3.02s/it]

이디야 매장 갯수 : 498


In [ ]:
'''
<li class="item">
    <a href="#c" onclick="panLatTo('0','0','0'); fnMove();">
        <div class="store_thum">
            <img src="../images/customer/store_thum.gif" />
        </div>
        <dl>
            <dt>강남YMCA점</dt>
            <dd>서울 강남구 논현동</dd>
        </dl>
    </a>
</li>
'''

In [93]:
edColumns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
edDataFrame = pd.DataFrame(ediyaData)
edDataFrame.columns = edColumns
edDataFrame.head()

,브랜드,상호,주소,시도,군구,위도,경도
0,이디야,동부제일병원삼거리점,"서울 중랑구 망우로 505 (망우동, 미래학원)",서울,중랑구,37.6006721423547,127.108627207598
1,이디야,동원사거리점,"서울 중랑구 겸재로 240 (면목동, 행복오피스텔)",서울,중랑구,37.5896269575279,127.094182772191
2,이디야,망우중앙점,"서울 중랑구 용마산로115길 109 (망우동, 한일써너스빌리젠시2단지)",서울,중랑구,37.5974674047065,127.09415879594557
3,이디야,망우코레일점,"서울 중랑구 망우로55길 11-10 (상봉동, 망우역)",서울,중랑구,37.5992876153903,127.092756577852
4,이디야,먹골역점,서울 중랑구 동일로157길 13 (묵동),서울,중랑구,37.6097411262067,127.076897926489


In [103]:
print('주소 변환에 문제가 있던 항목들')
incorrect_address

주소 변환에 문제가 있던 항목들


['서울 동작구 사당로16가길 96 (사당동) 1,2층', '서울 영등포구 영등포로35길 19 (영등포동6가)']

In [104]:
# 콤마를 공백으로, 2칸 공백은 1칸으로 치환합니다.
# 이후 공백으로 분리후, 슬라이싱을 사용하여 0번째 부터 3번째 글자까지만 추출합니다.
correct_address = [addr.replace(',', ' ').replace('  ', ' ').split(' ')[0:4] for addr in incorrect_address]
correct_address

[['서울', '동작구', '사당로16가길', '96'], ['서울', '영등포구', '영등포로35길', '19']]

In [105]:
# split() 함수로 나누어진 주소를 join() 함수로 다시 문자열을 합칩니다.
correct_address = [' '.join(addr) for addr in correct_address]
correct_address

['서울 동작구 사당로16가길 96', '서울 영등포구 영등포로35길 19']

In [106]:
# 보정이 된 주소 이름으로 Kakao API에게 위도/경도 추출을 요청합니다.
correction_geoinfo = [getGeoCoder(addr) for addr in correct_address]
correction_geoinfo

list index out of range


[('37.485107611205', '126.968823424325'), None]

In [94]:
edFilename = dataOut + 'ediya_file.csv'
edDataFrame.to_csv(edFilename, index=False, encoding='UTF-8')
print(f'{edFilename} 파일이 저장되었습니다.')

./../dataOut/ediya_file.csv 파일이 저장되었습니다.


In [95]:
# 위도, 경도 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {edDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {edDataFrame["경도"].isnull().sum()}')

위도 누락 데이터 갯수 : 2
경도 누락 데이터 갯수 : 2
